In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
contaminantes_2016 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2016.csv")
contaminantes_2017 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2017.csv")
contaminantes_2018 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2018.csv")
contaminantes_2019 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2019.csv")
contaminantes_2020 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2020.csv")
contaminantes_2021 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2021.csv")
contaminantes_2022 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2022.csv")
contaminantes_2023 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2023.csv")
contaminantes_2024 = pd.read_csv("../Datos/raw/contaminantes/contaminantes_2024.csv")

meteorologia_2016 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2016.csv")
meteorologia_2017 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2017.csv", header=10)
meteorologia_2018 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2018.csv", header=10)
meteorologia_2019 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2019.csv", header=10)
meteorologia_2020 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2020.csv", header=10)
meteorologia_2021 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2021.csv")
meteorologia_2022 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2022.csv", header=10)
meteorologia_2023 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2023.csv", header=10)
meteorologia_2024 = pd.read_csv("../Datos/raw/meteorologia/meteorologia_2024.csv")

Detalles en fechas de contaminantes:
- De 2016-2021 las horas incluyen valores de 01-24 de 01/01/año 01:00 - 31/12/año 24:00. **Solución**: convertir 24 en 00 del siguiente día
- De 2022 entre 23:00 y 01:00 hay NaNs y en formato año-mes-dia hora:min:seg. **Solución**: sustituir NaNs por 00 del día siguiente
- 2023 tenemos 00:00 y acaba en el 1 de enero de 2024 a las 00:00 y en formato año-mes-dia hora:min:seg.. **Cambiar formato de fecha**
- 2024 no tenemos ni 00:00 ni 24:00 si no solo en formato año-mes-dia hora:min:seg. **Cambiar formato de fecha**

In [3]:
def change_24(date_val):
    """
    If date_val is a string with hour == 24, increase the day by 1 and set hour to 00:00.
    Otherwise, return the original value.
    """
    if isinstance(date_val, str):
        try:
            date_part, time_part = date_val.strip().split()
            day, month, year = map(int, date_part.split('/'))
            hour, minute = map(int, time_part.split(':'))
            if hour == 24:
                dt = datetime(year, month, day) + timedelta(days=1)
                return dt.strftime('%d/%m/%Y 00:00')
            else:
                return date_val
        except Exception:
            return date_val
    return date_val

def impute_nan_dates(date_series):
    """
    Impute NaN values in a pandas Series of date strings.
    For each NaN, take the last valid date, add one day, and set time to 00:00:00.
    Assumes non-NaN values are in "%Y-%m-%d %H:%M:%S" format.
    Returns a new pandas Series with imputed values.
    """
    dates = date_series.copy()
    for i in range(len(date_series)):
        if isinstance(date_series[i], float) and np.isnan(date_series[i]):
            prev = dates[i-1]
            dt = datetime.strptime(prev, "%Y-%m-%d %H:%M:%S") + timedelta(days=1)
            dates[i] = dt.strftime("%Y-%m-%d 00:00:00")
    return dates

In [4]:
contaminantes_2016['date'] = contaminantes_2016['date'].apply(change_24)
contaminantes_2017['date'] = contaminantes_2017['date'].apply(change_24)
contaminantes_2018['date'] = contaminantes_2018['date'].apply(change_24)
contaminantes_2019['date'] = contaminantes_2019['date'].apply(change_24)
contaminantes_2020['date'] = contaminantes_2020['date'].apply(change_24)
contaminantes_2021['date'] = contaminantes_2021['date'].apply(change_24)
contaminantes_2022["date"] = impute_nan_dates(contaminantes_2022["date"])
contaminantes_2022["date"] = pd.to_datetime(contaminantes_2022["date"],
                                format = "%Y-%m-%d %H:%M:%S").dt.strftime('%d/%m/%Y %H:%M')
contaminantes_2023['date'] = pd.to_datetime(contaminantes_2023['date'],
                                format = "%Y-%m-%d %H:%M:%S").dt.strftime('%d/%m/%Y %H:%M')
# Fill missing hour/minute/second with 00:00:00 before formatting
contaminantes_2024['date'] = contaminantes_2024['date'].apply(
    lambda x: x if len(str(x).strip().split()) > 1 else f"{x} 00:00:00"
)
contaminantes_2024['date'] = pd.to_datetime(contaminantes_2024['date'],
                                format = "%Y-%m-%d %H:%M:%S").dt.strftime('%d/%m/%Y %H:%M')

In [5]:
dataframes = [contaminantes_2016, contaminantes_2017, contaminantes_2018, contaminantes_2019,
              contaminantes_2020, contaminantes_2021, contaminantes_2022, contaminantes_2023,
              contaminantes_2024]
df = pd.concat(dataframes)

df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y %H:%M')

In [6]:
print(df.shape)
df.head()

(20332550, 5)


,date,id_station,id_parameter,value,unit
0,2016-01-01 01:00:00,ACO,CO,NaN,15
1,2016-01-01 01:00:00,ACO,NO,NaN,1
2,2016-01-01 01:00:00,ACO,NO2,NaN,1
3,2016-01-01 01:00:00,ACO,NOX,NaN,1
4,2016-01-01 01:00:00,ACO,O3,NaN,1


In [7]:
data = df.groupby(["date","id_parameter"])["value"].mean().reset_index()

In [8]:
pivoted_df = data.pivot(index='date', columns='id_parameter', values='value')
pivoted_df = pivoted_df.reset_index()

In [9]:
pivoted_df

id_parameter,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PMCO,SO2
0,2016-01-01 01:00:00,1.338889,16.052632,46.210526,62.263158,9.739130,152.000000,81.785714,36.111111,18.294118
1,2016-01-01 02:00:00,1.494444,18.473684,46.315789,64.842105,7.739130,175.631579,111.857143,44.333333,21.176471
2,2016-01-01 03:00:00,1.444444,24.368421,43.000000,67.368421,6.652174,178.473684,127.928571,46.222222,31.647059
3,2016-01-01 04:00:00,1.383333,26.684211,41.000000,67.842105,6.000000,166.894737,133.500000,41.444444,39.235294
4,2016-01-01 05:00:00,1.383333,32.000000,39.315789,71.473684,4.434783,168.842105,135.285714,36.333333,41.176471
...,...,...,...,...,...,...,...,...,...,...
78898,2024-12-31 19:00:00,0.901667,3.650000,47.142857,52.500000,51.240000,73.125000,34.384615,24.857143,3.428571
78899,2024-12-31 20:00:00,0.650833,3.100000,35.666667,39.950000,41.840000,78.812500,26.615385,34.571429,2.904762
78900,2024-12-31 21:00:00,0.502500,2.700000,30.809524,33.900000,32.280000,59.933333,21.500000,32.833333,1.904762
78901,2024-12-31 22:00:00,0.554583,3.850000,32.666667,37.150000,24.000000,62.266667,23.083333,25.166667,1.857143


In [10]:
start = pd.Timestamp("2016-01-01 00:00")
end = pd.Timestamp("2024-12-31 23:00")
all_hours = set(pd.date_range(start, end, freq="h"))

missing_hours = all_hours - set(pivoted_df['date'])
missing_hours

{Timestamp('2016-01-01 00:00:00'),
 Timestamp('2024-01-17 19:00:00'),
 Timestamp('2024-04-07 00:00:00'),
 Timestamp('2024-04-07 02:00:00'),
 Timestamp('2024-04-07 03:00:00'),
 Timestamp('2024-04-07 04:00:00'),
 Timestamp('2024-04-07 05:00:00'),
 Timestamp('2024-04-07 06:00:00'),
 Timestamp('2024-04-07 07:00:00')}

In [11]:
pivoted_df.to_csv("../Datos/procesados/promedios_horas.csv", index=False)

In [12]:
data_max = df.groupby(["date","id_parameter"])["value"].max().reset_index()
pivoted_df_max = data_max.pivot(index='date', columns='id_parameter', values='value')
pivoted_df_max = pivoted_df_max.reset_index()

In [13]:
pivoted_df_max

id_parameter,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PMCO,SO2
0,2016-01-01 01:00:00,2.20,52.0,59.0,110.0,38.0,511.0,153.0,67.0,52.0
1,2016-01-01 02:00:00,2.50,58.0,62.0,111.0,29.0,308.0,214.0,93.0,84.0
2,2016-01-01 03:00:00,2.40,70.0,54.0,120.0,26.0,329.0,232.0,97.0,98.0
3,2016-01-01 04:00:00,1.90,54.0,51.0,102.0,19.0,258.0,187.0,71.0,117.0
4,2016-01-01 05:00:00,2.00,64.0,46.0,106.0,19.0,342.0,186.0,54.0,81.0
...,...,...,...,...,...,...,...,...,...,...
78898,2024-12-31 19:00:00,1.98,13.0,87.0,99.0,90.0,118.0,63.0,37.0,10.0
78899,2024-12-31 20:00:00,1.41,15.0,68.0,79.0,84.0,170.0,53.0,63.0,7.0
78900,2024-12-31 21:00:00,1.27,10.0,54.0,64.0,59.0,108.0,45.0,59.0,8.0
78901,2024-12-31 22:00:00,1.13,16.0,48.0,56.0,43.0,104.0,46.0,31.0,4.0


In [14]:
pivoted_df_max.to_csv("../Datos/procesados/maximos_horas.csv", index=False)

## Creación de media movil